# Fabric Item Management

## Imports & Constants

In [87]:
import requests
import time
import base64
import json
import os

from abc import ABC, abstractmethod
from uuid import uuid4
from typing import Optional, Generic, TypeVar, Any, Type, Self, Iterator, Callable
from dataclasses import dataclass, asdict, field


StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 89, Finished, Available, Finished)

In [88]:
WORKSPACE_ID = "b1ccaa7d-dbba-4158-b98e-790aa7205600"
TOKEN = notebookutils.credentials.getToken("https://api.fabric.microsoft.com")
HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}
WORKSPACE_URL = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_ID}"

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 90, Finished, Available, Finished)

## Utils

In [89]:
def base64_encode(obj: dict | str | bytes | bytearray) -> str:
    """
    Encodiert dicts (als JSON), Strings (UTF-8) oder rohe Bytes/Bytearray
    zu Base64 (ASCII-String).
    """
    if isinstance(obj, dict):
        obj = json.dumps(obj, ensure_ascii=True).encode("utf-8")
    elif isinstance(obj, str):
        obj = obj.encode("utf-8")
    elif isinstance(obj, (bytes, bytearray)):
        obj = bytes(obj)
    else:
        raise TypeError(f"Unsupported type: {type(obj)!r}")

    return base64.b64encode(obj).decode("ascii")


def base64_encode_zip(filepath: str) -> str:
    with open(filepath, "rb") as zf:
        zip_bytes = zf.read()
    return base64_encode(zip_bytes)


def base64_decode(obj_str: str):
    decoded = base64.b64decode(obj_str).decode("utf-8")
    return decoded

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 91, Finished, Available, Finished)

In [90]:
class ItemDefinitionInterface(ABC):
    @abstractmethod
    def get_definition(self) -> dict: pass

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 92, Finished, Available, Finished)

In [91]:
class FabricAPIWorkspaceClient:
    def __init__(self, client):
        self._client = client
        self._base_url = f"{client.base_url}/workspaces"

    def get(self, workspace_id: str, item_path: str) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.get(url, headers=self._client.headers)
        return resp

    def post(self, workspace_id: str, item_path: str, payload: dict) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.post(
            url,
            headers=self._client.headers,
            json=payload
        )
        return resp

    def patch(self, workspace_id: str, item_path: str, payload: dict) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.patch(
            url,
            headers=self._client.headers,
            json=payload
        )
        return resp

    def put(self, workspace_id: str, item_path: str, payload: dict) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.put(
            url,
            headers=self._client.headers,
            json=payload
        )
        return resp

    def delete(self, workspace_id: str, item_path: str) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.delete(url, headers=self._client.headers)
        return resp

    def _url(self, workspace_id: str, item_path: str) -> str:
        item_path = self._client._prep_path(item_path)
        url = f"{self._base_url}/{workspace_id}{item_path}"
        return url


class FabricAPIClient:
    def __init__(
        self,
        auth_type: str = None,
        version: str = "v1"
    ):
        self._base_url = f"https://api.fabric.microsoft.com/{version}"
        self.refresh_headers()

        self._workspaces = FabricAPIWorkspaceClient(self)

    @property
    def base_url(self) -> str:
        return self._base_url

    @property
    def workspaces(self) -> FabricAPIWorkspaceClient:
        return self._workspaces

    @property
    def headers(self) -> dict:
        return self._headers

    def refresh_headers(self) -> None:
        self._headers = {
            "Authorization": f"Bearer {self._get_token()}",
            "Content-Type": "application/json"
        }

    def get(self, path: str) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.get(url, headers=self.headers)
        return resp

    def post(self, path: str, payload: dict) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.post(
            url,
            headers=self.headers,
            json=payload
        )
        return resp

    def patch(self, path: str, payload: dict) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.patch(
            url,
            headers=self.headers,
            json=payload
        )
        return resp

    def put(self, path: str, payload: dict) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.put(
            url,
            headers=self.headers,
            json=payload
        )
        return resp

    def delete(self, path: str) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.delete(url, headers=self.headers)
        return resp

    def _url(self, path: str) -> str:
        path = self._prep_path(path)
        url = f"{self._base_url}{path}"
        return url

    def _prep_path(self, path: str) -> str:
        prep_path = path if path.startswith("/") else f"/{path}"
        return prep_path

    def _get_token(self) -> str:
        token = notebookutils.credentials.getToken("https://api.fabric.microsoft.com") # TODO
        return token

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 93, Finished, Available, Finished)

In [92]:
client = FabricAPIClient()

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 94, Finished, Available, Finished)

## BaseWorkspaceItem Implementation

In [93]:
def retry_after(resp: requests.Response, retry_max_seconds: int = 5) -> int:
    return min(int(resp.headers.get("Retry-After", retry_max_seconds)), retry_max_seconds)


def http_wait_for_completion_after_202(resp: requests.Response, payload: dict = None, retry_max_seconds: int = 5, timeout: int = 90) -> requests.Response:
    print("CODE:", resp.status_code)
    print("HEADERS:", resp.headers)
    print("json:", resp.json())
    op_id = resp.headers["x-ms-operation-id"]
    op_location = resp.headers["Location"]
    retry = retry_after(resp, retry_max_seconds)
    print(f"Status=202, Operation ID: {op_id}, Location: {op_location}, Retry after: {retry}s")

    retry_sum = 0
    obj = None
    while True:
        time.sleep(retry)
        resp_retry = requests.get(op_location, headers=client.headers)

        if resp_retry.json()["status"] == "Succeeded":
            res = requests.get(resp_retry.headers["Location"], headers=client.headers)
            res.raise_for_status()
            obj = res.json()
            break

        retry = retry_after(resp_retry)
        retry_sum += retry
        if retry_sum > timeout:
            print(f"Timeout after {timeout}s")
            raise TimeoutError(f"Timeout while waiting for item creation. Payload: {payload}")

        print(f"Wait for more {retry}s")

    return obj

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 95, Finished, Available, Finished)

In [94]:
TItemAPIData = TypeVar("TItemAPIData")


@dataclass
class FabricItem(Generic[TItemAPIData]):
    fields: dict[str, Any]
    apiData: Optional[TItemAPIData] = None

    def __init__(self, apiData: Optional[TItemAPIData] = None, **fields: dict):
        self.fields = fields
        self.apiData = apiData


class BaseWorkspaceItem(Generic[TItemAPIData]):
    def __init__(
            self,
            create_type_fn: Callable,
            base_item_url: str,
            workspace_id: str,
            item: FabricItem[TItemAPIData]
    ):
        self._create_type_fn = create_type_fn
        self._base_item_url = base_item_url
        self._workspace_id = workspace_id
        self._item: FabricItem[TItemAPIData] = item

    @property
    def item(self) -> FabricItem[TItemAPIData]:
        return self._item

    @staticmethod
    def get_by_id(
            create_type_fn: Callable,
            workspace_id: str,
            base_item_url: str,
            id: str
    ) -> Any:
        item_path = f"{base_item_url}/{id}"
        resp = client.workspaces.get(workspace_id, item_path)
        resp.raise_for_status()
        item = resp.json()
        print("ITEM BY ID:", item)
        return create_type_fn(item)

    @staticmethod
    def get_by_name(
            create_type_fn: Callable,
            workspace_id: str,
            base_item_url: str,
            name: str
    ) -> Any:
        item_path = base_item_url
        resp = client.workspaces.get(workspace_id, item_path)
        resp.raise_for_status()
        for item in resp.json()["value"]:
            if item["displayName"] == name:
                return create_type_fn(item)
        raise LookupError(f"{cls.__name__} '{name}' not found")

    @staticmethod
    def list(workspace_id: str, base_item_url: str) -> Iterator[Any]:
        item_path = base_item_url
        resp = client.workspaces.get(workspace_id, item_path)
        resp.raise_for_status()
        for item in resp.json()["value"]:
            yield item

    def fetch(self) -> Self:
        if self._item.apiData is None:
            self._item.apiData = BaseWorkspaceItem.get_by_name(
                create_type_fn=self._create_type_fn,
                workspace_id=self._workspace_id,
                base_item_url=self._base_item_url,
                name=self._item.fields["displayName"]
            ).item.apiData
            return self
        
        self._item.apiData = BaseWorkspaceItem.get_by_id(
            create_type_fn=self._create_type_fn,
            workspace_id=self._workspace_id,
            base_item_url=self._base_item_url,
            id=self._item.apiData.id
        ).item.apiData
        return self

    def fetch_definition(self) -> dict:
        if self._item.apiData is None:
            self.fetch()
        url = f"{self._base_item_url}/{self._item.apiData.id}/getDefinition"
        resp = client.workspaces.post(self._workspace_id, url, payload={})
        resp.raise_for_status()
        if resp.status_code == 202:
            return http_wait_for_completion_after_202(resp, retry_max_seconds=1)
        return resp.json()

    def exists(self) -> bool:
        try:
            return self.fetch()._item.apiData is not None
        except Exception as e: # TODO: nur bei not found; nicht bei allen exp
            return False
        
    def create(self) -> None:
        item_path = self._base_item_url
        payload = self._item.fields
        resp = client.workspaces.post(
            workspace_id=self._workspace_id,
            item_path=item_path,
            payload=payload
        )
        if resp.status_code > 299:
            print(resp.url)
            print("ERR:", resp.json())
        resp.raise_for_status()
        item = resp.json()
        if resp.status_code == 202 and item is None:
            item = http_wait_for_completion_after_202(resp)

        print("item:", item)
        self._item.apiData = self._create_type_fn(item).item.apiData
        self.fetch()

    def create_if_not_exists(self) -> None:
        if self.exists():
            return
        self.create()

    def update(self) -> None:
        if self._item.apiData is None:
            self.fetch()

        item_path = f"{self._base_item_url}/{self._item.apiData.id}"
        payload = self._item.fields
        resp = client.workspaces.patch(
            workspace_id=self._workspace_id,
            item_path=item_path,
            payload=payload
        )
        resp.raise_for_status()
        item = resp.json()
        self._item.apiData = self._create_type_fn(item).item.apiData
        self.fetch()

    def delete(self) -> None:
        if self._item.apiData is None:
            self.fetch()

        item_path = f"{self._base_item_url}/{self._item.apiData.id}"
        resp = client.workspaces.delete(
            workspace_id=self._workspace_id,
            item_path=item_path
        )
        resp.raise_for_status()

    def __str__(self) -> str:
        item_str = str(self._item)
        item_str_total = (
            f"{self.__class__.__name__}("
            f"workspaceId='{self._workspace_id}', "
            f"item={item_str}"
        )
        return item_str_total

    def __repr__(self) -> str:
        return str(self)

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 96, Finished, Available, Finished)

In [95]:
class CopyItemDefinition(ItemDefinitionInterface):
    def __init__(self, workspace_id: str, id: str, item_uri_name: str):
        self._wid = workspace_id
        self._id = id
        self._item_uri_name = item_uri_name

    def get_definition(self) -> dict:
        url = f"/workspaces/{self._wid}/{self._item_uri_name}/{self._id}/getDefinition"
        resp = client.post(url, payload={})
        resp.raise_for_status()
        definition = None
        if resp.status_code == 202:
            definition = http_wait_for_completion_after_202(resp, retry_max_seconds=1)
        else:
            definition = resp.json()
        return definition["definition"]

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 97, Finished, Available, Finished)

## Pipeline

In [96]:
@dataclass
class DataPipelineAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: str
    type: str


class CopyDataPipelineDefinition(CopyItemDefinition):
    def __init__(self, workspace_id: str, pipeline_id: str):
        super().__init__(
            workspace_id=workspace_id,
            id=pipeline_id,
            item_uri_name="dataPipelines"
        )


class ZIPDataPipelineDefinition(ItemDefinitionInterface):
    def __init__(self, pipeline_json: str):
        self._pipeline_json = pipeline_json

    def get_definition(self) -> dict:
        pipeline_json_b64 = base64_encode(self._pipeline_json)
        platform_payload_b64 = base64_encode({
            "$schema": "https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json",
            "metadata": {
                "type": "DataPipeline",
                "displayName": "pipeline1",
                "description": "A Description"
            },
            "config": {
                "version": "2.0",
                "logicalId": "00000000-0000-0000-0000-000000000000"
            }
        })

        return {
            "parts": [
                {
                    "path": "pipeline-content.json",
                    "payload": pipeline_json_b64,
                    "payloadType": "InlineBase64"
                },
                {
                    "path": ".platform",
                    "payload": platform_payload_b64,
                    "payloadType": "InlineBase64"
                }
            ]
        }

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 98, Finished, Available, Finished)

In [97]:
class DataPipeline(BaseWorkspaceItem[DataPipelineAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/notebook/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        definition: ItemDefinitionInterface = None,
        api_data: DataPipelineAPIData = None
    ):
        definition = definition.get_definition() if isinstance(definition, ItemDefinitionInterface) else None
        description = description or "New Data Pipeline"
        item = FabricItem[DataPipelineAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            definition=definition,
            apiData=api_data
        )
        super().__init__(
            create_type_fn=DataPipeline.from_json,
            base_item_url="/dataPipelines",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "DataPipeline":
        kwargs = item.copy()
        api_data =DataPipelineAPIData(**kwargs)
        return DataPipeline(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> "DataPipeline":
        return BaseWorkspaceItem.get_by_name(
            create_fn=DataPipeline.from_json,
            workspace_id=workspace_id,
            base_item_url="/dataPipelines",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> "DataPipeline":
        return BaseWorkspaceItem.get_by_id(
            create_fn=DataPipeline.from_json,
            workspace_id=workspace_id,
            base_item_url="/dataPipelines",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[DataPipelineAPIData]:
        return [
            Notebook.from_json(item)
            for item in BaseWorkspaceItem.list(
                workspace_id=workspace_id,
                base_item_url="/dataPipelines"
            )
        ]

dp = DataPipeline(WORKSPACE_ID, name="datapipeline")
print(dp)

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 99, Finished, Available, Finished)

DataPipeline(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'datapipeline', 'description': 'New Data Pipeline', 'folderId': None, 'definition': None}, apiData=None)


In [98]:
name = f"DP_{uuid4().hex[:8]}"
print("NAME:", name)

cp_pipeline_id = "5cf5c242-eb95-4a6b-b1e6-92ed890b3de5"

definition = CopyDataPipelineDefinition(WORKSPACE_ID, cp_pipeline_id)
dp_new = DataPipeline(
    WORKSPACE_ID,
    name=name,
    description="A Description",
    definition=definition
)

dp_new.create()
dp_new.delete()
# print(dp_new)

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 100, Finished, Available, Finished)

NAME: DP_3e678b70
item: {'id': 'f529d0be-bf65-428c-9305-9a7a3807a7a1', 'type': 'DataPipeline', 'displayName': 'DP_3e678b70', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': 'f529d0be-bf65-428c-9305-9a7a3807a7a1', 'type': 'DataPipeline', 'displayName': 'DP_3e678b70', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}


In [99]:
url = "https://github.com/enricogoerlitz/fabricengineer-py/raw/refs/heads/feature/api.fabric/notebooks/pipeline.json.txt"
resp = requests.get(url)

resp.raise_for_status()
pipeline_json = resp.json()

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 101, Finished, Available, Finished)

In [100]:
# print(base64_decode(dp_new.fetch_definition()["definition"]["parts"][0]["payload"]))

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 102, Finished, Available, Finished)

In [101]:
name = f"DP_{uuid4().hex[:8]}"
print("NAME:", name)

definition = ZIPDataPipelineDefinition(pipeline_json)
dp_new = DataPipeline(
    WORKSPACE_ID,
    name=name,
    description="A Description",
    definition=definition
)

dp_new.create()
# dp_new.delete()
# print(dp_new)

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 103, Finished, Available, Finished)

NAME: DP_1e4a6fd2
item: {'id': '3c4181bd-1d38-4eb0-909f-026c067b6000', 'type': 'DataPipeline', 'displayName': 'DP_1e4a6fd2', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '3c4181bd-1d38-4eb0-909f-026c067b6000', 'type': 'DataPipeline', 'displayName': 'DP_1e4a6fd2', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}


In [102]:
definition.get_definition()["parts"][0]

StatementMeta(, 70e7c2a4-946e-4af8-84b6-2fed3e6126b7, 104, Finished, Available, Finished)

{'path': 'pipeline-content.json',
 'payload': 'eyJwcm9wZXJ0aWVzIjogeyJhY3Rpdml0aWVzIjogW3sibmFtZSI6ICJJbXBvcnQgbWV0YWRhdGEiLCAidHlwZSI6ICJMb29rdXAiLCAiZGVwZW5kc09uIjogW10sICJwb2xpY3kiOiB7InRpbWVvdXQiOiAiMC4xMjowMDowMCIsICJyZXRyeSI6IDEsICJyZXRyeUludGVydmFsSW5TZWNvbmRzIjogMzAsICJzZWN1cmVPdXRwdXQiOiBmYWxzZSwgInNlY3VyZUlucHV0IjogZmFsc2V9LCAidHlwZVByb3BlcnRpZXMiOiB7InNvdXJjZSI6IHsidHlwZSI6ICJKc29uU291cmNlIiwgInN0b3JlU2V0dGluZ3MiOiB7InR5cGUiOiAiQXp1cmVCbG9iU3RvcmFnZVJlYWRTZXR0aW5ncyIsICJyZWN1cnNpdmUiOiBmYWxzZSwgImVuYWJsZVBhcnRpdGlvbkRpc2NvdmVyeSI6IGZhbHNlfSwgImZvcm1hdFNldHRpbmdzIjogeyJ0eXBlIjogIkpzb25SZWFkU2V0dGluZ3MifX0sICJkYXRhc2V0U2V0dGluZ3MiOiB7ImFubm90YXRpb25zIjogW10sICJ0eXBlIjogIkpzb24iLCAidHlwZVByb3BlcnRpZXMiOiB7ImxvY2F0aW9uIjogeyJ0eXBlIjogIkF6dXJlQmxvYlN0b3JhZ2VMb2NhdGlvbiIsICJmaWxlTmFtZSI6IHsidmFsdWUiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy5JbXBvcnRNZXRhZGF0YUZpbGVuYW1lIiwgInR5cGUiOiAiRXhwcmVzc2lvbiJ9LCAiZm9sZGVyUGF0aCI6IHsidmFsdWUiOiAiQGNvbmNhdChcbiAgICBwaXBlbGluZSgpLmxpYnJhcnlWYXJpYWJsZ

## Notebook

In [217]:
@dataclass
class NotebookAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: str
    type: str


class IPYNBNotebook(ItemDefinitionInterface):
    def __init__(self, code: str):
        self._code = code

    def get_definition(self) -> dict:
        code_b64 = base64_encode(self._code)
        platform_payload_b64 = base64_encode({
            "$schema": "https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json",
            "metadata": {
                "type": "Notebook",
                "displayName": "notebook",
                "description": "New Notebook"
            },
            "config": {
                "version": "2.0",
                "logicalId": "00000000-0000-0000-0000-000000000000"
            }
        })

        return {
            "format": "ipynb",
            "parts": [
                {
                    "path": "notebook-content.ipynb",
                    "payload": code_b64,
                    "payloadType": "InlineBase64"
                },
                {
                    "path": ".platform",
                    "payload": platform_payload_b64,
                    "payloadType": "InlineBase64"
                }
            ]
        }


class CopyFabricNotebook(ItemDefinitionInterface):
    def __init__(self, workspace_id: str, notebook_id: str):
        self._workspace_id = workspace_id
        self._notebook_id = notebook_id

    def get_definition(self) -> dict:
        url = f"/workspaces/{self._workspace_id}/notebooks/{self._notebook_id}/getDefinition"
        resp = client.post(url, payload={})
        print(resp.url)
        resp.raise_for_status()
        definition = http_wait_for_completion_after_202(resp, retry_max_seconds=1)
        return definition["definition"]


StatementMeta(, 8b338274-8f94-4a2e-a0be-abae8dc4abf9, 21, Finished, Available, Finished)

In [218]:
class Notebook(BaseWorkspaceItem[NotebookAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/notebook/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        fabric_notebook: ItemDefinitionInterface = None,
        api_data: NotebookAPIData = None
    ):
        definition = fabric_notebook.get_definition() if fabric_notebook else None
        description = description or "New Notebook"
        item = FabricItem[NotebookAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            definition=definition,
            apiData=api_data
        )
        super().__init__(
            create_type_fn=Notebook.from_json,
            base_item_url="/notebooks",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "Notebook":
        kwargs = item.copy()
        api_data = NotebookAPIData(**kwargs)
        return Notebook(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> "Notebook":
        return BaseWorkspaceItem.get_by_name(
            create_fn=Notebook.from_json,
            workspace_id=workspace_id,
            base_item_url="/notebooks",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> "Notebook":
        return BaseWorkspaceItem.get_by_id(
            create_fn=Notebook.from_json,
            workspace_id=workspace_id,
            base_item_url="/notebooks",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[NotebookAPIData]:
        return [
            Notebook.from_json(item)
            for item in BaseWorkspaceItem.list(
                workspace_id=workspace_id,
                base_item_url="/notebooks"
            )
        ]

nb = Notebook(WORKSPACE_ID, name="notebook")
print(nb)

StatementMeta(, 8b338274-8f94-4a2e-a0be-abae8dc4abf9, 22, Finished, Available, Finished)

Notebook(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'notebook', 'description': 'New Notebook', 'folderId': None, 'definition': None}, apiData=None)


In [219]:
df = spark.read.text("Files/notebooks/BASE SILVER NOTEBOOK v1.0.1.ipynb.txt")
# df now is a Spark DataFrame containing text data from "Files/notebooks/BASE SILVER NOTEBOOK v1.0.1.ipynb.txt".
code = "\n".join([r.value for r in df.collect()])
code = json.loads(code)
# print(code)

StatementMeta(, 8b338274-8f94-4a2e-a0be-abae8dc4abf9, 23, Finished, Available, Finished)

In [220]:
name = f"NB_{uuid4().hex[:8]}"
print("NAME:", name)

fabric_notebook = IPYNBNotebook(code)
nb_new = Notebook(
    WORKSPACE_ID,
    name=name,
    description="A Description",
    fabric_notebook=fabric_notebook
)

# json.loads(base64_decode(definition.definition["definition"]["parts"][1]["payload"]))
nb_new.create()
print(nb_new)

StatementMeta(, 8b338274-8f94-4a2e-a0be-abae8dc4abf9, 24, Finished, Available, Finished)

NAME: NB_80f62360


Status=202, Operation ID: 486a3be8-a574-4453-a670-fc05644c6bc7, Location: https://api.fabric.microsoft.com/v1/operations/486a3be8-a574-4453-a670-fc05644c6bc7, Retry after: 5s


item: {'id': '9b75d3b0-d518-4c42-b26d-01eb2e12e55b', 'type': 'Notebook', 'displayName': 'NB_80f62360', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '9b75d3b0-d518-4c42-b26d-01eb2e12e55b', 'type': 'Notebook', 'displayName': 'NB_80f62360', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
Notebook(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'NB_80f62360', 'description': 'A Description', 'folderId': None, 'definition': {'format': 'ipynb', 'parts': [{'path': 'notebook-content.ipynb', 'payload': 'eyJjZWxscyI6IFt7ImNlbGxfdHlwZSI6ICJtYXJrZG93biIsICJpZCI6ICIxNzU2MTJiYS01NTFmLTQ2ODktOTE0NC1jNGZhMWEwMjk2YzEiLCAibWV0YWRhdGEiOiB7Im1pY3Jvc29mdCI6IHsibGFuZ3VhZ2UiOiAicHl0aG9uIiwgImxhbmd1YWdlX2dyb3VwIjogInN5bmFwc2VfcHlzcGFyayJ9LCAibnRlcmFjdCI6IHsidHJhbnNpZW50IjogeyJkZWxldGluZyI6IGZhbHNlfX19LCAic291cmNlIjogWyIjIEJBU0UgUEFSQU0gU0lMVkVSIE5PVEVCT09LIHYxLjAuMCJdfS

In [224]:
name = f"NB_{uuid4().hex[:8]}"

fabric_notebook = CopyFabricNotebook(WORKSPACE_ID, "5241c6c7-63cc-40f1-87ba-9063353fbeca")
nb_new = Notebook(
    WORKSPACE_ID,
    name=name,
    description="A Description",
    fabric_notebook=fabric_notebook
)

# json.loads(base64_decode(definition.definition["definition"]["parts"][1]["payload"]))
nb_new.create()

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 226, Finished, Available, Finished)

https://api.fabric.microsoft.com/v1/workspaces/b1ccaa7d-dbba-4158-b98e-790aa7205600/notebooks/5241c6c7-63cc-40f1-87ba-9063353fbeca/getDefinition
REESP: None
Status=202, Operation ID: faff7725-737a-41c4-9dae-48d66ae4f6df, Location: https://api.fabric.microsoft.com/v1/operations/faff7725-737a-41c4-9dae-48d66ae4f6df, Retry after: 1s
DEFINITION: {'parts': [{'path': 'notebook-content.py', 'payload': 'IyBGYWJyaWMgbm90ZWJvb2sgc291cmNlCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAia2VybmVsX2luZm8iOiB7CiMgTUVUQSAgICAgIm5hbWUiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgICB9LAojIE1FVEEgICAiZGVwZW5kZW5jaWVzIjoge30KIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKIyBXZWxjb21lIHRvIHlvdXIgbmV3IG5vdGVib29rCiMgVHlwZSBoZXJlIGluIHRoZSBjZWxsIGVkaXRvciB0byBhZGQgY29kZSEKCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAibGFuZ3VhZ2UiOiAicHl0aG9uIiwKIyBNRVRBICAgImxhbmd1YWdlX2dyb3VwIjogInN5bmFwc2VfcHlzcGFyayIKIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKcHJpbnQoI

In [157]:
nb_new.create()

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 159, Finished, Available, Finished)

Status=202, Operation ID: c8478bd9-bdd0-402b-89ca-89046826bfa0, Location: https://api.fabric.microsoft.com/v1/operations/c8478bd9-bdd0-402b-89ca-89046826bfa0, Retry after: 5s
item: {'id': 'a18370f9-52dd-4b75-8428-54f36de3a55a', 'type': 'Notebook', 'displayName': 'NB_4367c143', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
DEFINITION: None
ITEM BY ID: {'id': 'a18370f9-52dd-4b75-8428-54f36de3a55a', 'type': 'Notebook', 'displayName': 'NB_4367c143', 'description': 'A Description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
DEFINITION: None


In [160]:
Notebook.list(WORKSPACE_ID)

StatementMeta(, 8b338274-8f94-4a2e-a0be-abae8dc4abf9, 28, Finished, Available, Finished)

[Notebook(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'Fabric Item Management', 'description': 'New notebook', 'folderId': None, 'definition': None}, apiData=NotebookAPIData(id='e1364ea9-b160-4eb9-8014-f21c68213cf3', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='Fabric Item Management', description='New notebook', type='Notebook')),
 Notebook(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'PlaygroundWarehouse', 'description': 'New Notebook', 'folderId': None, 'definition': None}, apiData=NotebookAPIData(id='34e7dd9b-8191-4010-90ef-18e4f85ef5a5', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='PlaygroundWarehouse', description='New Notebook', type='Notebook')),
 Notebook(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'notebook', 'description': 'New Notebook', 'folderId': None, 'definition': None}, apiData=NotebookAPIDa

In [126]:
client.workspaces.get(WORKSPACE_ID, "/notebooks").json()

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 128, Finished, Available, Finished)

{'value': [{'id': 'e1364ea9-b160-4eb9-8014-f21c68213cf3',
   'type': 'Notebook',
   'displayName': 'Fabric Item Management',
   'description': 'New notebook',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'},
  {'id': '34e7dd9b-8191-4010-90ef-18e4f85ef5a5',
   'type': 'Notebook',
   'displayName': 'PlaygroundWarehouse',
   'description': 'New Notebook',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'},
  {'id': '5241c6c7-63cc-40f1-87ba-9063353fbeca',
   'type': 'Notebook',
   'displayName': 'notebook',
   'description': 'New Notebook',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}]}

In [129]:
resp = client.workspaces.post(WORKSPACE_ID, "/notebooks/5241c6c7-63cc-40f1-87ba-9063353fbeca/getDefinition", payload={})
definition = http_wait_for_completion_after_202(resp, retry_max_seconds=1)
print(obj)

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 131, Finished, Available, Finished)

Status=202, Operation ID: 6075fca4-69ab-4ea7-a374-16bfecfc73f5, Location: https://api.fabric.microsoft.com/v1/operations/6075fca4-69ab-4ea7-a374-16bfecfc73f5, Retry after: 1s
{'definition': {'parts': [{'path': 'notebook-content.py', 'payload': 'IyBGYWJyaWMgbm90ZWJvb2sgc291cmNlCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAia2VybmVsX2luZm8iOiB7CiMgTUVUQSAgICAgIm5hbWUiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgICB9LAojIE1FVEEgICAiZGVwZW5kZW5jaWVzIjoge30KIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKIyBXZWxjb21lIHRvIHlvdXIgbmV3IG5vdGVib29rCiMgVHlwZSBoZXJlIGluIHRoZSBjZWxsIGVkaXRvciB0byBhZGQgY29kZSEKCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAibGFuZ3VhZ2UiOiAicHl0aG9uIiwKIyBNRVRBICAgImxhbmd1YWdlX2dyb3VwIjogInN5bmFwc2VfcHlzcGFyayIKIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKcHJpbnQoIkhlbGxvIFdvcmxkIikKCiMgTUVUQURBVEEgKioqKioqKioqKioqKioqKioqKioKCiMgTUVUQSB7CiMgTUVUQSAgICJsYW5ndWFnZSI6ICJweXRob24iLAojIE1FVEEgICAibGFuZ3VhZ2VfZ3JvdXAiOiA

In [72]:
nb1 = client.workspaces.get(WORKSPACE_ID, "/notebooks/5241c6c7-63cc-40f1-87ba-9063353fbeca").json()

resp = client.workspaces.post(WORKSPACE_ID, f"/notebooks/{nb1['id']}/getDefinition", payload={})
time.sleep(1)
resp = requests.get(resp.headers["Location"], headers=client.headers)
resp = requests.get(resp.headers["Location"], headers=client.headers)

definition = resp.json()["definition"]
definition

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 74, Finished, Available, Finished)

{'parts': [{'path': 'notebook-content.py',
   'payload': 'IyBGYWJyaWMgbm90ZWJvb2sgc291cmNlCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAia2VybmVsX2luZm8iOiB7CiMgTUVUQSAgICAgIm5hbWUiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgICB9LAojIE1FVEEgICAiZGVwZW5kZW5jaWVzIjoge30KIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKIyBXZWxjb21lIHRvIHlvdXIgbmV3IG5vdGVib29rCiMgVHlwZSBoZXJlIGluIHRoZSBjZWxsIGVkaXRvciB0byBhZGQgY29kZSEKCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAibGFuZ3VhZ2UiOiAicHl0aG9uIiwKIyBNRVRBICAgImxhbmd1YWdlX2dyb3VwIjogInN5bmFwc2VfcHlzcGFyayIKIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKcHJpbnQoIkhlbGxvIFdvcmxkIikKCiMgTUVUQURBVEEgKioqKioqKioqKioqKioqKioqKioKCiMgTUVUQSB7CiMgTUVUQSAgICJsYW5ndWFnZSI6ICJweXRob24iLAojIE1FVEEgICAibGFuZ3VhZ2VfZ3JvdXAiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgfQo=',
   'payloadType': 'InlineBase64'},
  {'path': '.platform',
   'payload': 'ewogICIkc2NoZW1hIjogImh0dHBzOi8vZGV2ZWxvcGVyLm1pY3Jvc29mdC5jb20vanNvbi1zY

In [69]:
base64_decode("IyBGYWJyaWMgbm90ZWJvb2sgc291cmNlCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAia2VybmVsX2luZm8iOiB7CiMgTUVUQSAgICAgIm5hbWUiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgICB9LAojIE1FVEEgICAiZGVwZW5kZW5jaWVzIjoge30KIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKIyBXZWxjb21lIHRvIHlvdXIgbmV3IG5vdGVib29rCiMgVHlwZSBoZXJlIGluIHRoZSBjZWxsIGVkaXRvciB0byBhZGQgY29kZSEKCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAibGFuZ3VhZ2UiOiAicHl0aG9uIiwKIyBNRVRBICAgImxhbmd1YWdlX2dyb3VwIjogInN5bmFwc2VfcHlzcGFyayIKIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKcHJpbnQoIkhlbGxvIFdvcmxkIikKCiMgTUVUQURBVEEgKioqKioqKioqKioqKioqKioqKioKCiMgTUVUQSB7CiMgTUVUQSAgICJsYW5ndWFnZSI6ICJweXRob24iLAojIE1FVEEgICAibGFuZ3VhZ2VfZ3JvdXAiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgfQo=")

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 71, Finished, Available, Finished)

'# Fabric notebook source\n\n# METADATA ********************\n\n# META {\n# META   "kernel_info": {\n# META     "name": "synapse_pyspark"\n# META   },\n# META   "dependencies": {}\n# META }\n\n# CELL ********************\n\n# Welcome to your new notebook\n# Type here in the cell editor to add code!\n\n\n# METADATA ********************\n\n# META {\n# META   "language": "python",\n# META   "language_group": "synapse_pyspark"\n# META }\n\n# CELL ********************\n\nprint("Hello World")\n\n# METADATA ********************\n\n# META {\n# META   "language": "python",\n# META   "language_group": "synapse_pyspark"\n# META }\n'

In [ ]:
df = spark.read.text("Files/notebooks/BASE SILVER NOTEBOOK v1.0.1.ipynb.txt")
# df now is a Spark DataFrame containing text data from "Files/notebooks/BASE SILVER NOTEBOOK v1.0.1.ipynb.txt".
ipynb_code = "\n".join([r.value for r in df.collect()])

base64_encode(ipynb_code)

In [36]:
s = base64_decode(definition["parts"][1]["payload"])
print(s)

StatementMeta(, e929248f-e34a-40af-ab38-70d78b05ac74, 38, Finished, Available, Finished)

{
  "$schema": "https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json",
  "metadata": {
    "type": "Notebook",
    "displayName": "notebook",
    "description": "New Notebook"
  },
  "config": {
    "version": "2.0",
    "logicalId": "00000000-0000-0000-0000-000000000000"
  }
}


## VariableLibrary

In [ ]:
client.workspaces.get(WORKSPACE_ID, "/VariableLibraries").json()

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 221, Finished, Available, Finished)

{'value': [{'id': '312682a8-935c-4fe4-9dca-be458217deb2',
   'type': 'VariableLibrary',
   'displayName': 'Environment Variables',
   'description': '',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600',
   'properties': {'activeValueSetName': 'Default value set'}},
  {'id': '55a1955c-1f99-46a0-9b12-c482189a8976',
   'type': 'VariableLibrary',
   'displayName': 'Environment Var-1',
   'description': 'No description',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600',
   'properties': {'activeValueSetName': 'Default value set'}},
  {'id': 'aab4328b-8c26-4deb-9acb-ebfc659cadbc',
   'type': 'VariableLibrary',
   'displayName': 'Environment Var-4-u',
   'description': 'No description',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600',
   'properties': {'activeValueSetName': 'Default value set'}},
  {'id': '685ee539-3c63-4065-a088-2f609c0fa7c7',
   'type': 'VariableLibrary',
   'displayName': 'Environment Var-10',
   'description': 'No description',
   'workspaceId': '

In [ ]:
var0 = client.workspaces.get(WORKSPACE_ID, "/VariableLibraries").json()["value"][0]


resp = client.workspaces.post(WORKSPACE_ID, f"/VariableLibraries/{var0['id']}/getDefinition", payload={})
time.sleep(1)
resp = requests.get(resp.headers["Location"], headers=client.headers)
resp = requests.get(resp.headers["Location"], headers=client.headers)

definition = resp.json()["definition"]
definition

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 222, Finished, Available, Finished)

{'parts': [{'path': 'variables.json',
   'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi92YXJpYWJsZXMvMS4wLjAvc2NoZW1hLmpzb24iLA0KICAidmFyaWFibGVzIjogWw0KICAgIHsNCiAgICAgICJuYW1lIjogIlZhcmlhYmxlMSIsDQogICAgICAibm90ZSI6ICIiLA0KICAgICAgInR5cGUiOiAiU3RyaW5nIiwNCiAgICAgICJ2YWx1ZSI6ICJibHViLWRlZmF1bHQiDQogICAgfSwNCiAgICB7DQogICAgICAibmFtZSI6ICJWYXJpYWJsZTIiLA0KICAgICAgIm5vdGUiOiAiIiwNCiAgICAgICJ0eXBlIjogIlN0cmluZyIsDQogICAgICAidmFsdWUiOiAidmFyMiINCiAgICB9DQogIF0NCn0=',
   'payloadType': 'InlineBase64'},
  {'path': 'settings.json',
   'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi9zZXR0aW5ncy8xLjAuMC9zY2hlbWEuanNvbiIsDQogICJ2YWx1ZVNldHNPcmRlciI6IFsNCiAgICAiVEVTVCIsDQogICAgIlBST0QiDQogIF0NCn0=',
   'payloadType': 'InlineBase64'},
  {'path': 'valueSets/TEST.json',
   'payload': 'ew0KICAiJHNjaGVtYSI

In [ ]:
parts = definition["parts"]
parts

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 223, Finished, Available, Finished)

[{'path': 'variables.json',
  'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi92YXJpYWJsZXMvMS4wLjAvc2NoZW1hLmpzb24iLA0KICAidmFyaWFibGVzIjogWw0KICAgIHsNCiAgICAgICJuYW1lIjogIlZhcmlhYmxlMSIsDQogICAgICAibm90ZSI6ICIiLA0KICAgICAgInR5cGUiOiAiU3RyaW5nIiwNCiAgICAgICJ2YWx1ZSI6ICJibHViLWRlZmF1bHQiDQogICAgfSwNCiAgICB7DQogICAgICAibmFtZSI6ICJWYXJpYWJsZTIiLA0KICAgICAgIm5vdGUiOiAiIiwNCiAgICAgICJ0eXBlIjogIlN0cmluZyIsDQogICAgICAidmFsdWUiOiAidmFyMiINCiAgICB9DQogIF0NCn0=',
  'payloadType': 'InlineBase64'},
 {'path': 'settings.json',
  'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi9zZXR0aW5ncy8xLjAuMC9zY2hlbWEuanNvbiIsDQogICJ2YWx1ZVNldHNPcmRlciI6IFsNCiAgICAiVEVTVCIsDQogICAgIlBST0QiDQogIF0NCn0=',
  'payloadType': 'InlineBase64'},
 {'path': 'valueSets/TEST.json',
  'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2Rl

In [ ]:
@dataclass
class VariableLibraryProperties:
    activeValueSetName: Optional[str] = None


@dataclass
class VariableLibraryAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: Optional[str]
    type: Optional[str]
    properties: VariableLibraryProperties = field(default_factory=VariableLibraryProperties)

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 224, Finished, Available, Finished)

In [ ]:
@dataclass
class VariableLibraryVariable:
    name: str
    note: str
    type: str
    value: Any


def _b64(obj: dict) -> str:
    json_bytes = json.dumps(obj, ensure_ascii=True).encode("utf-8")
    obj = base64.b64encode(json_bytes).decode("ascii")
    return obj


class VariableLibraryDefinition:
    def __init__(self, variable_lib_name: str, value_sets_ordered: list[str], *variables: list[VariableLibraryVariable]) -> None:
        self._var_lib_name = variable_lib_name
        self._value_sets = value_sets_ordered
        self._variables = variables or []

    @property
    def name(self) -> str:
        return self._var_lib_name

    def to_definition(self) -> list[dict]:
        variables = [asdict(var) for var in self._variables]
        variables_data = {
            "$schema": f"https://developer.microsoft.com/json-schemas/fabric/item/variableLibrary/definition/variables/1.0.0/schema.json",
            "variables": variables
        }
        settings_data = {
            "$schema": f"https://developer.microsoft.com/json-schemas/fabric/item/variableLibrary/definition/settings/1.0.0/schema.json",
            "valueSetsOrder": self._value_sets
        }
        platform_data = {
            "$schema": f"https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json",
            "metadata": {
                "type": "VariableLibrary",
                "displayName": self._var_lib_name
            },
            "config": {
                "version": "2.0",
                "logicalId": "00000000-0000-0000-0000-000000000000"
            }
        }

        variable_sets_data_b64 = [
            {
                "path": f"valueSets/{value_set}.json",
                "payload": _b64({
                    "$schema": "https://developer.microsoft.com/json-schemas/fabric/item/variableLibrary/definition/valueSet/1.0.0/schema.json",
                    "name": value_set,
                    "variableOverrides": [
                        {"name": var["name"], "value": var["value"]}
                        for var in variables
                    ]
                }),
                "payloadType": "InlineBase64"
            }
            for value_set in self._value_sets
        ]

        variables_data_b64 = _b64(variables_data)
        settings_data_b64 = _b64(settings_data)
        platform_data_b64 = _b64(platform_data)

        parts = [
            {
                "path": "variables.json",
                "payload": variables_data_b64,
                "payloadType": "InlineBase64"
            },
            {
                "path": "settings.json",
                "payload": settings_data_b64,
                "payloadType": "InlineBase64"
            }
        ] + variable_sets_data_b64 + [
            {
                "path": ".platform",
                "payload": platform_data_b64,
                "payloadType": "InlineBase64"
            }
        ]

        return {
            "parts": parts
        }

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 225, Finished, Available, Finished)

In [ ]:
class VariableLibrary(BaseWorkspaceItem[VariableLibraryAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/variablelibrary/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        definition: VariableLibraryDefinition = None,
        api_data: VariableLibraryAPIData = None
    ):
        definition = definition.to_definition() if isinstance(definition, VariableLibraryDefinition) else None
        description = description or "New VariableLibrary"
        item = FabricItem[VariableLibraryAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            definition=definition,
            apiData=api_data
        )
        super().__init__(
            create_type_fn=VariableLibrary.from_json,
            base_item_url="/VariableLibraries",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "VariableLibraryAPIData":
        kwargs = item.copy()
        if not "properties" in item.keys():
            item["properties"] = {}
        kwargs["properties"] = VariableLibraryProperties(**item["properties"])
        api_data = VariableLibraryAPIData(**kwargs)
        return VariableLibrary(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> VariableLibraryAPIData:
        return BaseWorkspaceItem.get_by_name(
            create_fn=VariableLibrary.from_json,
            workspace_id=workspace_id,
            base_item_url="/VariableLibraries",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> VariableLibraryAPIData:
        return BaseWorkspaceItem.get_by_id(
            create_fn=VariableLibrary.from_json,
            workspace_id=workspace_id,
            base_item_url="/VariableLibraries",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[VariableLibraryAPIData]:
        return [
            VariableLibrary.from_json(item)
            for item in BaseWorkspaceItem.list(
                workspace_id=workspace_id,
                base_item_url="/VariableLibraries"
            )
        ]


varlib = VariableLibrary(
    WORKSPACE_ID,
    name="VariableLibe"
)
print(varlib)

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 226, Finished, Available, Finished)

VariableLibrary(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'VariableLibe', 'description': 'New VariableLibrary', 'folderId': None, 'definition': None}, apiData=None)


In [ ]:
name = f"VL_{uuid4().hex[:10]}"

definition = VariableLibraryDefinition(
    name,
    ["TEST", "PROD"],
    VariableLibraryVariable(
        name="Variable1",
        note="",
        type="String",
        value="blub-default"
    ),
    VariableLibraryVariable(
        name="Variable2",
        note="",
        type="String",
        value="var2"
    )
)

varlib_new = VariableLibrary(
    workspace_id=WORKSPACE_ID,
    name=name,
    definition=definition
)

print(name)
varlib_new.create()
# varlib_new.delete()

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 228, Finished, Available, Finished)

Status=202, wait for success, retry=20 OP=6bac7ef7-6b74-40da-ace9-d778541a3337


WAIT JSON: {'status': 'Succeeded', 'createdTimeUtc': '2025-08-16T18:39:07.4905283', 'lastUpdatedTimeUtc': '2025-08-16T18:39:10.3967931', 'percentComplete': 100, 'error': None}
item: {'id': '4ba20307-eda8-440d-9963-d79e96f2e804', 'type': 'VariableLibrary', 'displayName': 'VL_90df91f885', 'description': 'New VariableLibrary', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '4ba20307-eda8-440d-9963-d79e96f2e804', 'type': 'VariableLibrary', 'displayName': 'VL_90df91f885', 'description': 'New VariableLibrary', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'activeValueSetName': 'Default value set'}}


## Warehouse

In [7]:
@dataclass
class WarehouseProperties:
    connectionInfo: Optional[str] = None
    connectionString: Optional[str] = None
    createdDate: Optional[str] = None
    creationMode: Optional[str] = None
    sourceRestorePoint: Optional[str] = None
    collationType: Optional[str] = None
    lastUpdatedTime: Optional[str] = None


@dataclass
class WarehouseAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: Optional[str]
    type: str
    properties: WarehouseProperties = field(default_factory=WarehouseProperties)


StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 87, Finished, Available, Finished)

In [8]:
class Warehouse(BaseWorkspaceItem[WarehouseAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/warehouse/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        collation_type: str = "Latin1_General_100_BIN2_UTF8",
        api_data: WarehouseAPIData = None
    ):
        description = description or "New Warehouse"
        item = FabricItem[WarehouseAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            creationPayload={
                "collationType": collation_type
            },
            apiData=api_data
        )
        super().__init__(
            create_type_fn=Warehouse.from_json,
            base_item_url="/warehouses",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "WarehouseAPIData":
        kwargs = item.copy()
        if not "properties" in item.keys():
            item["properties"] = {}
        kwargs["properties"] = WarehouseProperties(**item["properties"])
        api_data = WarehouseAPIData(**kwargs)
        return Warehouse(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> WarehouseAPIData:
        return BaseWorkspaceItem.get_by_name(
            create_fn=Warehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/warehouse",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> WarehouseAPIData:
        return BaseWorkspaceItem.get_by_id(
            create_fn=Warehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/warehouse",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[WarehouseAPIData]:
        return [
            Warehouse.from_json(item)
            for item in BaseWorkspaceItem.list(
                workspace_id=workspace_id,
                base_item_url="/warehouse"
            )
        ]


wh = Warehouse(WORKSPACE_ID, name="PlaygroundWarehouse")
print(wh)

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 88, Finished, Available, Finished)

Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'PlaygroundWarehouse', 'description': 'New Warehouse', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=None)


In [9]:
import time

name = f"WH_{uuid4().hex[:10]}"
wh_new = Warehouse(WORKSPACE_ID, name=name)
wh_new.create()
print(f"CREATED: {name}")
wh_new

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 89, Finished, Available, Finished)

Status=202, wait for success, retry=20 OP=d4b8637b-798e-4aaa-ae5d-161d1315e4b6


WAIT JSON: {'status': 'Succeeded', 'createdTimeUtc': '2025-08-16T15:16:24.2435824', 'lastUpdatedTimeUtc': '2025-08-16T15:16:36.5264967', 'percentComplete': 100, 'error': None}
item: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'connectionInfo': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'connectionString': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'createdDate': '2025-08-16T15:16:23.9935813', 'lastUpdatedTime': '2025-08-16T15:16:36.4015046', 'collationType': 'Latin1_General_100_BIN2_UTF8', 'creationMode': 'New', 'sourceRestorePoint': None}

Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=WarehouseAPIData(id='9b8c5121-e634-49de-be68-e77e4dc32ed7', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='WH_219a53ae10', description='New Warehouse', type='Warehouse', properties=WarehouseProperties(connectionInfo='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', createdDate='2025-08-16T15:16:23.9935813', creationMode='New', sourceRestorePoint=None, collationType='Latin1_General_100_BIN2_UTF8', lastUpdatedTime='2025-08-16T15:16:36.4015046')))

In [ ]:
wh_new.fetch()

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 90, Finished, Available, Finished)

ITEM BY ID: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'connectionInfo': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'connectionString': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'createdDate': '2025-08-16T15:16:23.9935813', 'lastUpdatedTime': '2025-08-16T15:16:36.4015046', 'collationType': 'Latin1_General_100_BIN2_UTF8', 'creationMode': 'New', 'sourceRestorePoint': None}}


Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=WarehouseAPIData(id='9b8c5121-e634-49de-be68-e77e4dc32ed7', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='WH_219a53ae10', description='New Warehouse', type='Warehouse', properties=WarehouseProperties(connectionInfo='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', createdDate='2025-08-16T15:16:23.9935813', creationMode='New', sourceRestorePoint=None, collationType='Latin1_General_100_BIN2_UTF8', lastUpdatedTime='2025-08-16T15:16:36.4015046')))

In [ ]:
wh_new.item.fields["description"] = "Updated description"
wh_new.update()

wh_new

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 91, Finished, Available, Finished)

ITEM BY ID: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'Updated description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'connectionInfo': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'connectionString': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'createdDate': '2025-08-16T15:16:23.9935813', 'lastUpdatedTime': '2025-08-16T15:18:20.2625873', 'collationType': 'Latin1_General_100_BIN2_UTF8', 'creationMode': 'New', 'sourceRestorePoint': None}}


Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'WH_219a53ae10', 'description': 'Updated description', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=WarehouseAPIData(id='9b8c5121-e634-49de-be68-e77e4dc32ed7', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='WH_219a53ae10', description='Updated description', type='Warehouse', properties=WarehouseProperties(connectionInfo='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', createdDate='2025-08-16T15:16:23.9935813', creationMode='New', sourceRestorePoint=None, collationType='Latin1_General_100_BIN2_UTF8', lastUpdatedTime='2025-08-16T15:18:20.2625873')))

In [ ]:
wh_new.delete()

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 92, Finished, Available, Finished)

## Folder

In [55]:
"""
curl -X POST \
  "https://api.fabric.microsoft.com/v1/workspaces/${WORKSPACE_ID}/folders" \
  -H "Authorization: Bearer $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{
        "displayName": "Data",
        "parentFolderId": null
      }'
"""

resp = client.workspaces.post(WORKSPACE_ID, "/folders", {
    "displayName": "newFolder",
    "parentFolderId": None
})

print(resp.status_code)
print(resp.text)

StatementMeta(, 94472719-572d-4b40-b18f-ed0020b12ce7, 57, Finished, Available, Finished)

201
{"id":"04fe8565-87d9-47c5-bc0a-d41957ad660e","displayName":"newFolder","workspaceId":"b1ccaa7d-dbba-4158-b98e-790aa7205600"}


In [56]:
resp = client.workspaces.get(WORKSPACE_ID, "/folders")
print(resp.text)

StatementMeta(, 94472719-572d-4b40-b18f-ed0020b12ce7, 58, Finished, Available, Finished)

{"value":[{"id":"04fe8565-87d9-47c5-bc0a-d41957ad660e","displayName":"newFolder","workspaceId":"b1ccaa7d-dbba-4158-b98e-790aa7205600"}]}


In [25]:
@dataclass
class WorkspaceFolderAPIData:
    id: str
    workspaceId: str
    displayName: str
    parentFolderId: Optional[str] = None

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 27, Finished, Available, Finished)

In [26]:
class WorkspaceFolder(BaseWorkspaceItem[WorkspaceFolderAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/?
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        folder_id: str = None,
        api_data: WorkspaceFolderAPIData = None
    ):
        item = FabricItem[WorkspaceFolderAPIData](
            displayName=name,
            parentFolderId=folder_id,
            apiData=api_data
        )
        super().__init__(
            create_type_fn=WorkspaceFolder.from_json,
            base_item_url="/folders",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "WorkspaceFolder":
        kwargs = item.copy()
        api_data = WorkspaceFolderAPIData(**kwargs)
        return WorkspaceFolder(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            api_data=api_data
        )

    
    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> "WorkspaceFolder":
        return BaseWorkspaceItem.get_by_name(
            create_fn=WorkspaceFolder.from_json,
            workspace_id=workspace_id,
            base_item_url="/folders",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> "WorkspaceFolder":
        return BaseWorkspaceItem.get_by_id(
            create_fn=WorkspaceFolder.from_json,
            workspace_id=workspace_id,
            base_item_url="/folders",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[WorkspaceFolderAPIData]:
        return [
            WorkspaceFolder.from_json(item)
            for item in BaseWorkspaceItem.list(
                workspace_id=workspace_id,
                base_item_url="/folders"
            )
        ]


item = WorkspaceFolder(WORKSPACE_ID, name="newFolder")
print(item)

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 28, Finished, Available, Finished)

WorkspaceFolder(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'newFolder', 'parentFolderId': None}, apiData=None)


In [27]:
item.fetch()

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 29, Finished, Available, Finished)

WorkspaceFolder(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'newFolder', 'parentFolderId': None}, apiData=WorkspaceFolderAPIData(id='04fe8565-87d9-47c5-bc0a-d41957ad660e', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='newFolder', parentFolderId=None))

In [28]:
item_new = WorkspaceFolder(WORKSPACE_ID, name=f"GreateSubFolder_{uuid4().hex[:4]}", folder_id=item.item.apiData.id)
item_new.create()

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 30, Finished, Available, Finished)

item: {'id': 'd0f9e325-50d3-487d-acad-6517a1380e34', 'displayName': 'GreateSubFolder_338d', 'parentFolderId': '04fe8565-87d9-47c5-bc0a-d41957ad660e', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': 'd0f9e325-50d3-487d-acad-6517a1380e34', 'displayName': 'GreateSubFolder_338d', 'parentFolderId': '04fe8565-87d9-47c5-bc0a-d41957ad660e', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}


## Lakehouse

In [9]:
@dataclass
class LakehouseSqlEndpointProperties:
    id: Optional[str] = None
    connectionString: Optional[str] = None
    provisioningStatus: Optional[str] = None


@dataclass
class LakehouseProperties:
    defaultSchema: Optional[str] = None
    oneLakeTablesPath: Optional[str] = None
    oneLakeFilesPath: Optional[str] = None
    sqlEndpointProperties: LakehouseSqlEndpointProperties = field(default_factory=LakehouseSqlEndpointProperties)


@dataclass
class LakehouseAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: Optional[str]
    type: str
    properties: LakehouseProperties = field(default_factory=LakehouseProperties)


StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 11, Finished, Available, Finished)

In [12]:
class Lakehouse(BaseWorkspaceItem[LakehouseAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/lakehouse/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        enable_schemas: bool = True,
        api_data: LakehouseAPIData = None
    ):
        description = description or "New Lakehouse"
        item = FabricItem[LakehouseAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            creationPayload={
                "enableSchemas": enable_schemas
            },
            apiData=api_data
        )
        super().__init__(
            create_type_fn=Lakehouse.from_json,
            base_item_url="/lakehouses",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "LakehouseAPIData":
        kwargs = item.copy()
        if not "properties" in item.keys():
            item["properties"] = {
                "sqlEndpointProperties": {}
            }
        kwargs["properties"] = LakehouseProperties(**item["properties"])
        kwargs["properties"].sqlEndpointProperties = LakehouseSqlEndpointProperties(
            **item["properties"]["sqlEndpointProperties"]
        )
        api_data = LakehouseAPIData(**kwargs)
        return Lakehouse(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    
    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> LakehouseAPIData:
        return BaseWorkspaceItem.get_by_name(
            create_fn=Lakehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/lakehouses",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> LakehouseAPIData:
        return BaseWorkspaceItem.get_by_id(
            create_fn=Lakehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/lakehouses",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[LakehouseAPIData]:
        return [
            Lakehouse.from_json(item)
            for item in BaseWorkspaceItem.list(
                workspace_id=workspace_id,
                base_item_url="/lakehouses"
            )
        ]


lh = Lakehouse(WORKSPACE_ID, name="PlaygroundLakehouse")
print(lh)

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 14, Finished, Available, Finished)

Lakehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'PlaygroundLakehouse', 'description': 'New Lakehouse', 'folderId': None, 'creationPayload': {'enableSchemas': True}}, apiData=None)


In [13]:
lh.fetch()

print(lh)

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 15, Finished, Available, Finished)

Lakehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'PlaygroundLakehouse', 'description': 'New Lakehouse', 'folderId': None, 'creationPayload': {'enableSchemas': True}}, apiData=LakehouseAPIData(id='deb7937b-6bbf-4bfb-9e45-9a29ccefad25', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='PlaygroundLakehouse', description='', type='Lakehouse', properties=LakehouseProperties(defaultSchema='dbo', oneLakeTablesPath='https://onelake.dfs.fabric.microsoft.com/b1ccaa7d-dbba-4158-b98e-790aa7205600/deb7937b-6bbf-4bfb-9e45-9a29ccefad25/Tables', oneLakeFilesPath='https://onelake.dfs.fabric.microsoft.com/b1ccaa7d-dbba-4158-b98e-790aa7205600/deb7937b-6bbf-4bfb-9e45-9a29ccefad25/Files', sqlEndpointProperties=LakehouseSqlEndpointProperties(id='844eb85c-412b-47d9-ae1a-9b9d13365f0e', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', provisioningStatus='Success'))))


In [14]:
lh.item.apiData.properties.sqlEndpointProperties.provisioningStatus

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 16, Finished, Available, Finished)

'Success'

In [17]:
import time


lh_new = Lakehouse(WORKSPACE_ID, name=f"LH_{uuid4().hex[:10]}")
lh_new.create()
print("CREATED")

while lh_new.fetch().item.apiData.properties.sqlEndpointProperties.provisioningStatus != "Success":
    state = lh_new.item.apiData.properties.sqlEndpointProperties.provisioningStatus
    print("CURRENT STATE:", state)
    time.sleep(2.5)


state = lh_new.item.apiData.properties.sqlEndpointProperties.provisioningStatus
print("CURRENT STATE:", state)
lh_new

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 19, Finished, Available, Finished)

item: {'id': '4b44d733-2bda-4dec-a522-f71b501134c5', 'type': 'Lakehouse', 'displayName': 'LH_db204d2c4e', 'description': 'New Lakehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '4b44d733-2bda-4dec-a522-f71b501134c5', 'type': 'Lakehouse', 'displayName': 'LH_db204d2c4e', 'description': 'New Lakehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'oneLakeTablesPath': 'https://onelake.dfs.fabric.microsoft.com/b1ccaa7d-dbba-4158-b98e-790aa7205600/4b44d733-2bda-4dec-a522-f71b501134c5/Tables', 'oneLakeFilesPath': 'https://onelake.dfs.fabric.microsoft.com/b1ccaa7d-dbba-4158-b98e-790aa7205600/4b44d733-2bda-4dec-a522-f71b501134c5/Files', 'sqlEndpointProperties': {'connectionString': None, 'id': None, 'provisioningStatus': 'InProgress'}, 'defaultSchema': 'dbo'}}
CREATED
ITEM BY ID: {'id': '4b44d733-2bda-4dec-a522-f71b501134c5', 'type': 'Lakehouse', 'displayName': 'LH_db204d2c4e', 'description': 'New Lakehouse', 'workspaceId': 'b1ccaa7d-

Lakehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'LH_db204d2c4e', 'description': 'New Lakehouse', 'folderId': None, 'creationPayload': {'enableSchemas': True}}, apiData=LakehouseAPIData(id='4b44d733-2bda-4dec-a522-f71b501134c5', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='LH_db204d2c4e', description='New Lakehouse', type='Lakehouse', properties=LakehouseProperties(defaultSchema='dbo', oneLakeTablesPath='https://onelake.dfs.fabric.microsoft.com/b1ccaa7d-dbba-4158-b98e-790aa7205600/4b44d733-2bda-4dec-a522-f71b501134c5/Tables', oneLakeFilesPath='https://onelake.dfs.fabric.microsoft.com/b1ccaa7d-dbba-4158-b98e-790aa7205600/4b44d733-2bda-4dec-a522-f71b501134c5/Files', sqlEndpointProperties=LakehouseSqlEndpointProperties(id='ed441628-3022-49a7-9c43-202e590c836b', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', provisioningStatus='Success'))))

In [ ]:
lh_new.fetch() # provisioningStatus InProgress -> noch nicht fertig!

In [ ]:
lh_new.item.fields["description"] = "Updated description"
lh_new.update()

lh_new

In [ ]:
lh_new.create_if_not_exists()

In [18]:
lh_new.delete()

StatementMeta(, 6b452751-2ff7-4ba5-8770-3e74b9cc5313, 20, Finished, Available, Finished)

In [ ]:
lakehouses = Lakehouse.list(WORKSPACE_ID)

len(lakehouses)